In [145]:
!pip install transformers

In [146]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [245]:
df = pd.read_csv("/content/drive/MyDrive/DATASETS/Telecom_bot_v4.csv")
df = df.dropna()

In [246]:
df.head()

,request,label
0,i want to recharge my mobile,Get_recharge
1,how to recharge my mobile,Get_recharge
2,how to recharge my sim,Get_recharge
3,i would like to recharge my mobile,Get_recharge
4,help me to recharge my mobile,Get_recharge


In [247]:
# from transformers import DistilBertTokenizer, TFDistilBertModel
# distil_bert = 'distilbert-base-uncased' 
# tokenizer = DistilBertTokenizer.from_pretrained(distil_bert)
# model = TFDistilBertModel.from_pretrained(distil_bert)

In [248]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

In [249]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [250]:
tokenized = df['request'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [251]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

In [252]:
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [253]:
np.array(padded).shape

(283, 14)

In [254]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(283, 14)

In [255]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
features = last_hidden_states[0][:,0,:].numpy()

In [256]:
features.shape

(283, 768)

In [257]:
bo = df['label']
bo.head()

0    Get_recharge
1    Get_recharge
2    Get_recharge
3    Get_recharge
4    Get_recharge
Name: label, dtype: object

In [258]:
# from sklearn.preprocessing import LabelBinarizer

In [259]:
# y = LabelBinarizer().fit_transform(bo)

In [260]:
features.shape

(283, 768)

In [261]:
labels = bo

In [262]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [263]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [264]:
lr_clf.score(test_features, test_labels)

0.9436619718309859

In [270]:
# text = "i would like to know about my current balance"
text = "it was fun talking to you see you again dude"
enc = tokenizer.encode(text)
max_len = len(enc)
padded = np.array([enc+[0]*(max_len-len(enc))])
attention_mask = np.where(padded != 0,1,0)
input_ids1 = torch.tensor(padded)  
attention_mask1 = torch.tensor(attention_mask)
# max_len
with torch.no_grad():
    last_hidden_states1 = model(input_ids1, attention_mask=attention_mask1)
a = last_hidden_states1[0].numpy()
b = a[:,0,:]
# features1 = last_hidden_states[0][:,0,:].numpy()
# features.shape
val = lr_clf.predict(b)
val

array(['New_connection'], dtype=object)